# 3. Kafka Consumer - Read Messages from Topic

Consumes messages from the sample topic in the MSK cluster. Run after the Producer notebook to read the messages.

In [ ]:
import json
import boto3
from kafka import KafkaConsumer

In [ ]:
# Get Kafka bootstrap servers from SSM
ssm = boto3.client('ssm')
bootstrap_servers = ssm.get_parameter(Name='/kafka/bootstrap_servers', WithDecryption=True)['Parameter']['Value']
print(f"Bootstrap servers: {bootstrap_servers}")

In [ ]:
TOPIC = 'sample-topic'

# Create consumer (reads from earliest, stops after 5s of no new messages)
consumer = KafkaConsumer(
    TOPIC,
    bootstrap_servers=bootstrap_servers.split(','),
    auto_offset_reset='earliest',
    value_deserializer=lambda m: json.loads(m.decode('utf-8')),
    key_deserializer=lambda k: k.decode('utf-8') if k else None,
    consumer_timeout_ms=5000
)

## Consume and print messages

In [ ]:
print(f"Consuming from topic '{TOPIC}'...\n")
messages = list(consumer)
print(f"Received {len(messages)} messages:\n")
for msg in messages:
    print(f"  partition={msg.partition} offset={msg.offset} key={msg.key}")
    print(f"  value: {msg.value}\n")
consumer.close()

## Quick consume (single cell - run after producing)

In [ ]:
consumer = KafkaConsumer(TOPIC, bootstrap_servers=bootstrap_servers.split(','),
    auto_offset_reset='earliest', consumer_timeout_ms=5000,
    value_deserializer=lambda m: json.loads(m.decode('utf-8')))
for msg in consumer:
    print(msg.value)
consumer.close()